MIT License

Copyright (c) Microsoft Corporation. All rights reserved.

This notebook is adapted from Francesca Lazzeri Energy Demand Forecast Workbench workshop.

Copyright (c) 2021 PyLadies Amsterdam, Alyona Galyeva

# Linear regression with recursive feature elimination

In [1]:
%matplotlib inline
import os
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy as np
from typing import Callable
# from azureml.core import Workspace, Dataset
# from azureml.core.experiment import Experiment
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import *
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


This notebook shows how to train a linear regression model to create a forecast of future energy demand. In particular, the model will be trained to predict energy demand in period $t_{+1}$, one hour ahead of the current time period $t$. This is known as 'one-step' time series forecasting because we are predicting one period into the future.

In [2]:
# os.mkdir('Linear_models')
# os.mkdir('NaiveBayes_models')

In [3]:
WORKDIR = os.getcwd()
TRAIN_DIR = os.path.join(WORKDIR, '../data-processing/data/train')
TEST_DIR = os.path.join(WORKDIR, '../data-processing/data/test')

In [4]:
# assign all the training data into DFs
for train_data in os.listdir(TRAIN_DIR):
    locals()['df_' + train_data[5:-4]] = pd.read_csv(TRAIN_DIR + '/' + train_data)
    print(f'The local variable of type DataFrame, df_{train_data[5:-4]} is created')
# df_train_1.head()

The local variable of type DataFrame, df_train_4 is created
The local variable of type DataFrame, df_train_3 is created
The local variable of type DataFrame, df_train_2 is created
The local variable of type DataFrame, df_train_1 is created
The local variable of type DataFrame, df_outlier_train_4 is created
The local variable of type DataFrame, df_outlier_train_3 is created
The local variable of type DataFrame, df_outlier_train_2 is created
The local variable of type DataFrame, df_outlier_train_1 is created


In [5]:
# assign all the testing data into DFs
for test_data in os.listdir(TEST_DIR):
    locals()['df_' + test_data[5:-4]] = pd.read_csv(TEST_DIR + '/' + test_data)
    print(f'The local variable of type DataFrame, df_{test_data[5:-4]} is created')
# df_test_1.head()

The local variable of type DataFrame, df_test_4 is created
The local variable of type DataFrame, df_test_1 is created
The local variable of type DataFrame, df_test_2 is created
The local variable of type DataFrame, df_test_3 is created
The local variable of type DataFrame, df_outlier_test_4 is created
The local variable of type DataFrame, df_outlier_test_1 is created
The local variable of type DataFrame, df_outlier_test_3 is created
The local variable of type DataFrame, df_outlier_test_2 is created


Create design matrix - each column in this matrix represents a model feature and each row is a training example. We remove the *demand* and *timeStamp* variables as they are not model features.

In [6]:
# # Create a function that can retutn all the DataFrames of the training & testing variables
# # making it easier to modify all the variables at once. This

# def modify_all_df(function: Callable) -> None:
#     for df_key in locals().keys():
#         if 'data_' in df_key:
#             # locals[df_key].apply(lambda x: function)
#             function(locals()[df_key])

In [7]:
import re
train_regex = re.compile(r'df_(\w+_)?train_\d')
df_train_list = list(filter(train_regex.match, locals().keys()))
df_train_list

['df_train_4',
 'df_train_3',
 'df_train_2',
 'df_train_1',
 'df_outlier_train_4',
 'df_outlier_train_3',
 'df_outlier_train_2',
 'df_outlier_train_1']

In [8]:
# set the X for every training data, by dropping the 'timestamp' & 'local_actual_mv' columns
# set the y for every training data, by only selecting the 'load_actual_mw'
X_train_list = []
for train_df in df_train_list:
        locals()[f'X_{train_df[3:]}'] = locals()[train_df].drop(['timestamp', 'load_actuals_mw'], axis = 1)
        X_train_list.append(f'X_{train_df[3:]}')
        print(f'The varaible, X_{train_df[3:]} is created.')

        locals()[f'y_{train_df[3:]}'] = locals()[train_df]['load_actuals_mw']
        print(f'The varaible, y_{train_df[3:]} is created.')

The varaible, X_train_4 is created.
The varaible, y_train_4 is created.
The varaible, X_train_3 is created.
The varaible, y_train_3 is created.
The varaible, X_train_2 is created.
The varaible, y_train_2 is created.
The varaible, X_train_1 is created.
The varaible, y_train_1 is created.
The varaible, X_outlier_train_4 is created.
The varaible, y_outlier_train_4 is created.
The varaible, X_outlier_train_3 is created.
The varaible, y_outlier_train_3 is created.
The varaible, X_outlier_train_2 is created.
The varaible, y_outlier_train_2 is created.
The varaible, X_outlier_train_1 is created.
The varaible, y_outlier_train_1 is created.


### Create predictive model pipeline

Here we use sklearn's Pipeline functionality to create a predictive model pipeline. For this model, the pipeline implements the following steps:
- **one-hot encode categorical variables** - this creates a feature for each unique value of a categorical feature. For example, the feature *dayofweek* has 7 unique values. This feature is split into 7 individual features dayofweek0, dayofweek1, ... , dayofweek6. The value of these features is 1 if the timeStamp corresponds to that day of the week, otherwise it is 0.
- **recursive feature elimination with cross validation (RFECV)** - it is often the case that some features add little predictive power to a model and may even make the model accuracy worse. Recursive feature elimination tests the model accuracy on increasingly smaller subsets of the features to identify the subset which produces the most accurate model. Cross validation is used to test each subset on multiple folds of the input data. The best model is that which achieves the lowest mean squared error averaged across the cross validation folds.
- **train final model** - the best model found in after the feature elimination process is used to train the final estimator on the whole dataset.

Identify indices for categorical columns for one hot encoding and create the OneHotEncoder:

In [9]:
# 改由在data-processing階段處理類別變數，以免train&test的feature數量不同

In [10]:
#categorical_cols = ['month', 'season']
#sample_df = locals()["df_train_1"] # Cuz all the DFs' structure are the same, so we'll only need one sample for the col index
#cat_cols_index = [sample_df.columns.get_loc(col) for col in sample_df.columns if col in categorical_cols]
#cat_cols_index

In [11]:
# transfer the categorical data into dummy variables
#for train_X in X_train_list:
#    locals()['dummy_' + train_X[2:]] = pd.get_dummies(data = locals()[train_X], columns = categorical_cols)
#    print(f"The variable, dummy_{train_X[2:]} is created")

In [12]:
lr = LinearRegression(fit_intercept=True)
lr

LinearRegression()

In [13]:
naive_bayes = GaussianNB()

For hyperparameter tuning and feature selection, cross validation will be performed using the training set. With time series forecasting, it is important that test data comes from a later time period than the training data. This also applies to each fold in cross validation. Therefore a time series split is used to create three folds for cross validation as illustrated below. Each time series plot represents a separate training/test split, with the training set coloured in blue and the test set coloured in red. Note that, even in the first split, the training data covers at least a full year so that the model can learn the annual seasonality of the demand.

In [14]:
tscv = TimeSeriesSplit(n_splits=3)

Create the RFECV object. Note the metric for evaluating the model on each fold is the negative mean squared error. The best model is that which maximises this metric.

In [15]:
regr_cv = RFECV(estimator=lr,
             cv=tscv,
             scoring='neg_mean_squared_error',
             verbose = 2,
             n_jobs = -1)

Create the model pipeline object.

In [16]:
lr_pipeline = Pipeline([('rfecv', regr_cv)])
lr_pipeline

Pipeline(steps=[('rfecv',
                 RFECV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                       estimator=LinearRegression(), n_jobs=-1,
                       scoring='neg_mean_squared_error', verbose=2))])

In [17]:
#naive bayes model的estimator應該要改成naive_bayes, 但改了後面model.pkl會報錯, 故先不動
nb_cv = RFECV(estimator=lr,
             cv=tscv,
             scoring='neg_mean_squared_error',
             verbose = 2,
             n_jobs = -1)

In [18]:
naive_bayes_pipeline = Pipeline([('rfecv', nb_cv)])
naive_bayes_pipeline

Pipeline(steps=[('rfecv',
                 RFECV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                       estimator=LinearRegression(), n_jobs=-1,
                       scoring='neg_mean_squared_error', verbose=2))])

In [19]:
# 改成存入model_pipeline才對
def save_model(model_pipeline: str, model_name: str, DIR: str) -> None:
    with open(os.path.join(DIR, model_name + '.pkl'), 'wb') as f:
        pickle.dump(model_pipeline, f)

In [20]:
#dummy_train_1

In [21]:
for i in range(1,5):
    try:
        # Linear Regression
        LR_DIR = os.path.join(WORKDIR + '/Linear_models')

        lr_pipeline.fit(locals()['X_train_' + str(i)].values, locals()['y_train_' + str(i)]) # for processed data
        save_model(lr_pipeline, 'lr_model_' + str(i), LR_DIR)

        lr_pipeline.fit(locals()['X_outlier_train_' + str(i)].values, locals()['y_outlier_train_' + str(i)]) # for outliers included data
        save_model(lr_pipeline, 'lr_outliers_model_' + str(i), LR_DIR)
    except Exception as e:
        with open(os.path.join(WORKDIR, 'error_handler.txt'), 'a') as f:
            f.write('< LR_model >\n')
            f.write(f'{e}\n')
            f.write('=================\n')

    try:
        # Naive Bayes
        NB_DIR = os.path.join(WORKDIR + '/NaiveBayes_models')

        naive_bayes_pipeline.fit(locals()['X_train_' + str(i)].values, locals()['y_train_' + str(i)])
        save_model(naive_bayes_pipeline, 'NaiveBayes_model_' + str(i), NB_DIR)

        naive_bayes_pipeline.fit(locals()['X_outlier_train_' + str(i)].values, locals()['y_outlier_train_' + str(i)])
        save_model(naive_bayes_pipeline, 'NaiveBayes_outliers_model_' + str(i), NB_DIR)
    except Exception as e:
        with open(os.path.join(WORKDIR, 'error_handler.txt'), 'a') as f:
            f.write('< NaiveBayes_model >\n')
            f.write(f'{e}\n')
            f.write('=================\n')

Fitting estimator with 45 features.
Fitting estimator with 45 features.
Fitting estimator with 44 features.
Fitting estimator with 44 features.
Fitting estimator with 43 features.
Fitting estimator with 43 features.
Fitting estimator with 42 features.
Fitting estimator with 42 features.
Fitting estimator with 41 features.
Fitting estimator with 41 features.
Fitting estimator with 40 features.
Fitting estimator with 39 features.
Fitting estimator with 40 features.
Fitting estimator with 38 features.
Fitting estimator with 39 features.
Fitting estimator with 37 features.
Fitting estimator with 38 features.
Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 37 features.
Fitting estimator with 34 features.
Fitting estimator with 36 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 35 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 34 fe

In [22]:
lr_pipeline.named_steps['rfecv'].n_features_

27

In [23]:
supported_features = pd.DataFrame.from_dict(
    {'feature':X_train_4.columns, 
     'supported':lr_pipeline.named_steps['rfecv'].support_}
)
supported_features

,feature,supported
0,temperature,True
1,solar_ghi,False
2,solar_prediction_mw,True
3,wind_prediction_mw,False
4,workdayornot,True
5,temperature_lag1,True
6,temperature_lag2,True
7,temperature_lag3,True
8,temperature_lag4,True
9,temperature_lag5,True
